In [ ]:
import pandas as pd
from sklearn import tree
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import cohen_kappa_score, roc_auc_score, f1_score
from statistics import mean
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

In [100]:
# Exploring the Dataset
data = pd.read_csv('ca1-dataset.csv')
print(data.head())

                                           Unique-id       namea OffTask  \
0  awagner-closeloop-ins_h1zaz4-03.30.2011_at_13:...  stuZgy46jl       N   
1  awagner-closeloop-ins_h1zaz4-03.30.2011_at_13:...  stuZ77be0l       N   
2  awagner-closeloop-ins_h1zaz4-03.30.2011_at_13:...  stuZ5lp7k7       N   
3  awagner-closeloop-ins_h1zaz4-03.30.2011_at_13:...  stuZ9idg5l       N   
4  awagner-closeloop-ins_h1zaz4-03.30.2011_at_13:...  stuZ4lil98       N   

   Avgright  Avgbug  Avghelp  Avgchoice  Avgstring  Avgnumber  Avgpoint  ...  \
0  1.000000     0.0        0          0          0          0         0  ...   
1  1.000000     0.0        0          0          0          0         0  ...   
2  1.000000     0.0        0          0          0          0         0  ...   
3  0.833333     0.0        0          0          0          0         0  ...   
4  1.000000     0.0        0          0          0          0         0  ...   

   Avghelppct-up  Avgwrongpct-up  Avgtimeperact-up  AvgPrev3Co

In [101]:
data['OffTask'] = data['OffTask'].replace({'Y': 1, 'N': 0})
X = data.drop(columns=['OffTask', 'Unique-id', 'namea'], axis=1)
y = data.OffTask

group_dict = {}
groups = np.array([])

for index, row in data.iterrows():
    student_id = row['namea']
    if student_id not in group_dict:
        group_dict[student_id] = index
    groups = np.append(groups, group_dict[student_id])

gkf = GroupKFold(n_splits=10)
kappa_values = []
f1_values = []
roc_auc_values = []
acc_values = []

In [102]:
# Decision Tree Classifier using 10 folds:
for train_index, test_index in gkf.split(X, y, groups=groups):
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    
    # Feature scaling (standardization)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    dtf = DecisionTreeClassifier(min_samples_split=10)
    dtf.fit(X_train_scaled, y_train)
    
    predictions = dtf.predict(X_test_scaled)
    
    kappa = cohen_kappa_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)
    roc_auc = roc_auc_score(y_test, dtf.predict_proba(X_test_scaled)[:, 1])
    accuracy = accuracy_score(y_test, predictions)
    
    kappa_values.append(kappa)
    f1_values.append(f1)
    roc_auc_values.append(roc_auc)
    acc_values.append(accuracy)

# Calculate final evaluation metrics
kappa_final = mean(kappa_values).round(2)
accuracy_final = np.mean(acc_values).round(2)
f1_final = np.mean(f1_values).round(2)
roc_auc_final = mean(roc_auc_values).round(2)

print("Using Decision Tree Classifier Method with 10 folds and feature scaling")
print(f"Cohen's Kappa: {kappa_final}")
print(f"F1: {f1_final}")
print(f"Accuracy: {accuracy_final}")
print(f"ROC_AUC: {roc_auc_final}")

Using Decision Tree Classifier Method with 10 folds and feature scaling
Cohen's Kappa: 0.17
F1: 0.2
Accuracy: 0.94
ROC_AUC: 0.68


In [103]:
# Decision Tree Classifier using 10 folds:
for train_index, test_index in gkf.split(X, y, groups=groups):
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    
    
    dtf = DecisionTreeClassifier(min_samples_split=10)
    dtf.fit(X_train, y_train)
    
    predictions = dtf.predict(X_test)
    
    kappa = cohen_kappa_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)
    roc_auc = roc_auc_score(y_test, dtf.predict_proba(X_test)[:, 1])
    accuracy = accuracy_score(y_test, predictions)
    
    kappa_values.append(kappa)
    f1_values.append(f1)
    roc_auc_values.append(roc_auc)
    acc_values.append(accuracy)

# Calculate final evaluation metrics
kappa_final = mean(kappa_values).round(2)
accuracy_final = np.mean(acc_values).round(2)
f1_final = np.mean(f1_values).round(2)
roc_auc_final = mean(roc_auc_values).round(2)

print("Using Decision Tree Classifier Method with 10 folds, without feature scaling")
print(f"Cohen's Kappa: {kappa_final}")
print(f"F1: {f1_final}")
print(f"Accuracy: {accuracy_final}")
print(f"ROC_AUC: {roc_auc_final}")

Using Decision Tree Classifier Method with 10 folds, without feature scaling
Cohen's Kappa: 0.19
F1: 0.21
Accuracy: 0.94
ROC_AUC: 0.68


In [104]:
gnb = GaussianNB()
# GNB Classifier using 10 folds:
for train_index, test_index in gkf.split(X, y, groups=groups):
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    
    # Feature scaling (standardization)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    gnb.fit(X_train_scaled, y_train)
    predictions = gnb.predict(X_test_scaled)
    
    kappa = cohen_kappa_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)
    roc_auc = roc_auc_score(y_test, gnb.predict_proba(X_test_scaled)[:, 1])
    accuracy = accuracy_score(y_test, predictions)
    
    kappa_values.append(kappa)
    f1_values.append(f1)
    roc_auc_values.append(roc_auc)
    acc_values.append(accuracy)


# Calculate final evaluation metrics
kappa_final = mean(kappa_values).round(2)
accuracy_final = np.mean(acc_values).round(2)
f1_final = np.mean(f1_values).round(2)
roc_auc_final = mean(roc_auc_values).round(2)

print("Using Gaussian Naive Bayes with 10 folds with feature scaling")
print(f"Cohen's Kappa: {kappa_final}")
print(f"F1: {f1_final}")
print(f"Accuracy: {accuracy_final}")
print(f"ROC_AUC: {roc_auc_final}")

Using Gaussian Naive Bayes with 10 folds with feature scaling
Cohen's Kappa: 0.17
F1: 0.2
Accuracy: 0.89
ROC_AUC: 0.66


In [105]:
gnb = GaussianNB()
# GNB Classifier using 10 folds:
for train_index, test_index in gkf.split(X, y, groups=groups):
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    
    gnb.fit(X_train, y_train)
    predictions = gnb.predict(X_test)
    
    kappa = cohen_kappa_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)
    roc_auc = roc_auc_score(y_test, gnb.predict_proba(X_test)[:, 1])
    accuracy = accuracy_score(y_test, predictions)
    
    kappa_values.append(kappa)
    f1_values.append(f1)
    roc_auc_values.append(roc_auc)
    acc_values.append(accuracy)


# Calculate final evaluation metrics
kappa_final = mean(kappa_values).round(2)
accuracy_final = np.mean(acc_values).round(2)
f1_final = np.mean(f1_values).round(2)
roc_auc_final = mean(roc_auc_values).round(2)

print("Using Gaussian Naive Bayes with 10 folds without Feature Scaling ")
print(f"Cohen's Kappa: {kappa_final}")
print(f"F1: {f1_final}")
print(f"Accuracy: {accuracy_final}")
print(f"ROC_AUC: {roc_auc_final}")

Using Gaussian Naive Bayes with 10 folds without Feature Scaling 
Cohen's Kappa: 0.15
F1: 0.19
Accuracy: 0.87
ROC_AUC: 0.65


In [106]:
for train_index, test_index in gkf.split(X, y, groups=groups):
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    
    # Feature scaling (standardization)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    xgbmodel = xgb.XGBClassifier(learning_rate=0.5, n_estimators=200, random_state=5)
    xgbmodel.fit(X_train_scaled, y_train)
    
    predictions = xgbmodel.predict(X_test_scaled)

    kappa = cohen_kappa_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)
    roc_auc = roc_auc_score(y_test, xgbmodel.predict_proba(X_test_scaled)[:, 1])
    accuracy = accuracy_score(y_test, predictions)
    
    kappa_values.append(kappa)
    f1_values.append(f1)
    roc_auc_values.append(roc_auc)
    acc_values.append(accuracy)


# Calculate final evaluation metrics
kappa_final = mean(kappa_values).round(2)
accuracy_final = np.mean(acc_values).round(2)
f1_final = np.mean(f1_values).round(2)
roc_auc_final = mean(roc_auc_values).round(2)

print("XGBoost with 10-fold, learning rate = 0.5 with feature scaling")
print(f"Cohen's Kappa: {kappa_final}")
print(f"F1: {f1_final}")
print(f"Accuracy: {accuracy_final}")
print(f"ROC_AUC: {roc_auc_final}")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if i

XGBoost with 10-fold, learning rate = 0.5 with feature scaling
Cohen's Kappa: 0.19
F1: 0.22
Accuracy: 0.89
ROC_AUC: 0.67


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


In [107]:
for train_index, test_index in gkf.split(X, y, groups=groups):
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    

    xgbmodel = xgb.XGBClassifier(learning_rate=0.5, n_estimators=200, random_state=5)
    xgbmodel.fit(X_train, y_train)
    
    predictions = xgbmodel.predict(X_test)

    kappa = cohen_kappa_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)
    roc_auc = roc_auc_score(y_test, xgbmodel.predict_proba(X_test)[:, 1])
    accuracy = accuracy_score(y_test, predictions)
    
    kappa_values.append(kappa)
    f1_values.append(f1)
    roc_auc_values.append(roc_auc)
    acc_values.append(accuracy)


# Calculate final evaluation metrics
kappa_final = mean(kappa_values).round(2)
accuracy_final = np.mean(acc_values).round(2)
f1_final = np.mean(f1_values).round(2)
roc_auc_final = mean(roc_auc_values).round(2)

print("XGBoost with 10-fold, learning rate = 0.5 without feature scaling")
print(f"Cohen's Kappa: {kappa_final}")
print(f"F1: {f1_final}")
print(f"Accuracy: {accuracy_final}")
print(f"ROC_AUC: {roc_auc_final}")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in

XGBoost with 10-fold, learning rate = 0.5 without feature scaling
Cohen's Kappa: 0.21
F1: 0.24
Accuracy: 0.9
ROC_AUC: 0.67


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in

In [116]:
for train_index, test_index in gkf.split(X, y, groups=groups):
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    
    # Feature scaling (standardization)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    xgbmodel = xgb.XGBClassifier(learning_rate=0.55, n_estimators=200, random_state=4)
    xgbmodel.fit(X_train_scaled, y_train)
    
    predictions = xgbmodel.predict(X_test_scaled)

    kappa = cohen_kappa_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)
    roc_auc = roc_auc_score(y_test, xgbmodel.predict_proba(X_test_scaled)[:, 1])
    accuracy = accuracy_score(y_test, predictions)
    
    kappa_values.append(kappa)
    f1_values.append(f1)
    roc_auc_values.append(roc_auc)
    acc_values.append(accuracy)


# Calculate final evaluation metrics
kappa_final = mean(kappa_values).round(2)
accuracy_final = np.mean(acc_values).round(2)
f1_final = np.mean(f1_values).round(2)
roc_auc_final = mean(roc_auc_values).round(2)

print("XGBoost with 10-fold, learning rate = 0.55, random_state = 4 without feature scaling")
print(f"Cohen's Kappa: {kappa_final}")
print(f"F1: {f1_final}")
print(f"Accuracy: {accuracy_final}")
print(f"ROC_AUC: {roc_auc_final}")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if i

XGBoost with 10-fold, learning rate = 0.55, random_state = 4 without feature scaling
Cohen's Kappa: 0.28
F1: 0.3
Accuracy: 0.94
ROC_AUC: 0.69


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
